In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['hike-hackathon-main', 'fork-of-hike-hackathon-main-dl-after-84-4', 'hike-hackathon']


In [2]:
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
json_file = open('../input/fork-of-hike-hackathon-main-dl-after-84-4/modeldl.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../input/fork-of-hike-hackathon-main-dl-after-84-4/modeldl.h5")

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
from keras.utils import to_categorical
from tqdm import tqdm

In [5]:
def make_matr(lst1, lst2):
    l1=np.asarray(lst1[1:-1].split(),dtype=int)
    l2=np.asarray(lst2[1:-1].split(),dtype=int)
    l1= np.concatenate((l1[0:-1],to_categorical(l1[-1],num_classes=16)), axis=0)
    l2= np.concatenate((l2[0:-1],to_categorical(l2[-1],num_classes=16)), axis=0)
    return np.concatenate((l1,l2))

In [6]:
data= pd.read_csv("../input/hike-hackathon/test_merged_uf.csv")

In [7]:
r1= len(data['node1'])/3
r2= len(data['node1'])*2/3


In [8]:
print(r1,r2)

3925656.0 7851312.0


In [9]:
data['is_chat']= None

In [10]:

def prediction_result(s,e):
    mat=[]
    for i in tqdm(range(s,e)):
        mat.append(make_matr(data['node1'][i], data['node2'][i]))
    mat= np.asmatrix(mat,dtype=int)
    ans=loaded_model.predict_proba(mat)
    return ans;
    

In [11]:
a1=prediction_result(0,int(r1))
a2=prediction_result(int(r1),int(r2))
a3=prediction_result(int(r2), len(data['node1']))

 87%|████████▋ | 3400933/3925656 [05:50<00:52, 9981.54it/s]

In [12]:
ans= np.concatenate((a1,a2,a3))

In [13]:
data['is_chat']=ans

In [14]:
# for i in tqdm(range(0,int(r1))):
#     data['is_chat'][i]=a1[i]
#     data['is_chat'][i+r1]=a2[i]
#     data['is_chat'][i+r2]=a3[i]

In [15]:
res= data[['id','is_chat']]

In [16]:
type(res)

pandas.core.frame.DataFrame

In [17]:
res['is_chat'].value_counts()

0.165836    117600
0.566956     93388
0.142026     40065
0.122536     37910
0.059196     29630
0.248221     12441
0.244868     11481
0.304619     11376
0.221399      9851
0.314893      9794
0.320064      9768
0.179065      7884
0.038149      7608
0.268970      7207
0.258442      7108
0.181079      7071
0.054540      6563
0.542911      6227
0.041506      5776
0.387891      5728
0.176616      5702
0.396147      5685
0.433902      5501
0.040334      5166
0.897085      5098
0.699428      5054
0.407498      4330
0.194034      4307
0.074652      4107
0.267759      4031
             ...  
0.224066         1
0.224084         1
0.444872         1
0.592214         1
0.804960         1
0.224085         1
0.223215         1
0.134048         1
0.166715         1
0.444873         1
0.224084         1
0.444874         1
0.224084         1
0.223214         1
0.224085         1
0.444874         1
0.224067         1
0.150879         1
0.036870         1
0.224086         1
0.310671         1
0.142157    

In [18]:
res.to_csv("Submission1.csv", sep=",", index=False)